In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import gym

In [3]:
class ActorCriticNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ActorCriticNN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Softmax(dim=1)
        )
     
    def forward(self, observation):
        return self.network(observation)

In [ ]:
env = gym.make('LunarLander-v2')

print("Possible actions:", env.action_space.n)
print("Possible states:")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device to be used:", device)

In [ ]:
actor = ActorCriticNN(env.observation_space[0], env.action_space.n).to(device)
critic = ActorCriticNN(env.observation_space[0], 1).to(device)

In [ ]:
# Hyperparameters
learning_rate = 0.00001
gamma = 0.99 # for calculating cumulative rewards
print_rate = 50 # rate that the output will be printed

In [ ]:
actor_optimizer = torh.optim.Adam(actor.parameters(), lr=learning_rate)
critic_optimizer = torh.optim.Adam(critic.parameters(), lr=learning_rate)

In [ ]:
def train(env, n_episodes=30):
